In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.models import Sequential

In [ ]:


train_dir='../input/inaturalist/inaturalist_12K/train'
test_dir='../input/inaturalist/inaturalist_12K/val'


In [ ]:
!pip install wandb
import wandb
wandb.login()

In [ ]:
# configure sweep parameters
sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "val_accuracy",
  "goal": "maximize"
  },
  "parameters": {
                   'base_filters': {'values': [32, 64]},
                   'filter_org': {'values': [0.5, 1, 2]}, 
                   'data_augment': {'values': [False, True]},
                   'batch_norm': {'values': [False, True]}, 
                   "batch_size": { "values": [32, 64] },
                   'pool_size' : {'values': [2]},
                   'dropout_rate': {'values': [0.0, 0.2, 0.3]},
                    'learning_rate':{ 'values': [0.0004 ]},
                   'kernel_size': {'values': [3, 5, 7]},
                   'dense_size': {'values': [64,512,1024]},
                   'activation': {'values': ['relu']},
                   'epochs': {'values': [10,20,30]}, 
                   'conv_layers': {'values': [5]}
                }
}

sweep_id = wandb.sweep(sweep_config,project='CS6910 Assignment2', entity='go4rav')

In [2]:
def train():

    # initialising wandb
    wandb.init()

    # loading configured parameters
    CONFIG=wandb.config
    base_filters = CONFIG.base_filters
    filter_org= CONFIG.filter_org
    dropout_rate = CONFIG.dropout_rate
    kernel_size = CONFIG.kernel_size
    pool_size = CONFIG.pool_size
    dense_size = CONFIG.dense_size
    data_augment = CONFIG.data_augment
    learning_rate = CONFIG.learning_rate
    batch_norm = CONFIG.batch_norm
    batch_size = CONFIG.batch_size
    epochs = CONFIG.epochs
    activation = CONFIG.activation
    conv_layers = CONFIG.conv_layers
    input_shape = (256,256,3)



    filters = [base_filters]

    # if filter organisation is same in each layer
    if filter_org == 1 :
      for i in range(1, conv_layers) :
        filters.append(filters[i - 1])

    # if filter organisation is halves after each layer
    elif filter_org == 0.5 :
      for i in range(1, conv_layers) :
        filters.append(filters[i - 1] / 2)
    
    # # if filter organisation is doubles after each layer
    elif filter_org == 2 :
      for i in range(1, conv_layers) :
        filters.append(filters[i - 1] * 2)

    wandb.run.name =  "_ft_"+str(base_filters)+"_fto_"+str(filter_org)+"_ksize_"+str(kernel_size)+"_dn_" + str(dense_size)+ "_dro_" + str(dropout_rate) + "_bs_"+str(batch_size)+"_da_"+str(data_augment)+"_bn_"+str(batch_norm)



    with tf.device('/device:GPU:0'):
      tf.keras.backend.clear_session()
      model = Sequential() 
      for i in range(0,conv_layers): 

          # adding convolution layer
          model.add(Conv2D(filters[i], kernel_size= (kernel_size,kernel_size), input_shape=input_shape, activation=activation))
          # adding max_pooling layer
          model.add(MaxPooling2D(pool_size=(pool_size,pool_size))) 
          # adding batch normalization
          if(batch_norm == True):
            model.add(BatchNormalization())


      # Flattening
      model.add(Flatten()) 

      # Adding a dense layer
      model.add(Dense(dense_size, activation=activation))

      # Adding batch normalsation
      if(batch_norm == True):
            model.add(BatchNormalization())

      # adding dropout 
      model.add(tf.keras.layers.Dropout(rate=dropout_rate))

      # adding output layer
      model.add(Dense(10, activation='softmax'))
      
      model.summary()
      img_height,img_width=(256,256)


      # data augmentation
      if data_augment == True:
        datagen= ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1.0 / 255,
        validation_split=0.1,
        )
      else:
        datagen = ImageDataGenerator(
        rescale=1.0 / 255,
        validation_split=0.1,
        )

      train_data = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      batch_size= batch_size,
      class_mode='categorical',
      shuffle=True,
      subset='training',
      seed=100,
      )

      valid_data = datagen.flow_from_directory(
      train_dir,
      target_size=(img_height, img_width),
      class_mode='categorical',
      shuffle=True,
      subset='validation',
      seed=100,
      )

      optimiser = tf.keras.optimizers.Adam(learning_rate=learning_rate)

      # model compilation
      model.compile(optimizer=optimiser, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

      # model fitting
      model.fit(train_data, epochs=epochs, validation_data=valid_data, callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 5),
                                            wandb.keras.WandbCallback()])

      # model saving
      model.save('./TrainedModel/'+wandb.run.name)
      wandb.finish()
 


In [1]:
wandb.agent(sweep_id, train ,count = 20)

# New Section